In [ ]:

import pandas as pd
import unittest
import logging
from typing import Any

class TradeSignal:
    BUY = 'BUY'
    SELL = 'SELL'
    HOLD = 'HOLD'

class Strategy:
    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def backtest_strategy(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Backtest the strategy with proper handling of single trades.
        """
        self.logger.info("Starting backtest with single-trade handling.")

        # Initialize tracking columns
        df['position'] = 0
        df['entry_price'] = 0.0
        df['exit_price'] = 0.0
        df['strategy_returns'] = 0.0
        df['equity'] = 100000.0  # Starting equity
        equity = 100000.0
        position = 0
        entry_price = 0.0

        for idx in range(len(df)):
            signal = df.at[idx, 'signal']
            price = df.at[idx, 'close']

            # Exit logic
            if position == 1 and signal == TradeSignal.SELL:  # Exiting LONG
                df.at[idx, 'exit_price'] = price
                pnl = price - entry_price
                equity += pnl
                df.at[idx, 'strategy_returns'] = pnl
                position = 0  # Reset position to flat
                entry_price = 0.0
                self.logger.debug(f"Exited LONG at index {idx} with equity {equity}")

            elif position == -1 and signal == TradeSignal.BUY:  # Exiting SHORT
                df.at[idx, 'exit_price'] = price
                pnl = entry_price - price
                equity += pnl
                df.at[idx, 'strategy_returns'] = pnl
                position = 0  # Reset position to flat
                entry_price = 0.0
                self.logger.debug(f"Exited SHORT at index {idx} with equity {equity}")

            # Entry logic (only when flat)
            if position == 0:
                if signal == TradeSignal.BUY:  # Enter LONG
                    position = 1
                    entry_price = price
                    df.at[idx, 'entry_price'] = entry_price
                    self.logger.debug(f"Entered LONG at index {idx}")
                elif signal == TradeSignal.SELL:  # Enter SHORT
                    position = -1
                    entry_price = price
                    df.at[idx, 'entry_price'] = entry_price
                    self.logger.debug(f"Entered SHORT at index {idx}")

            # Maintain the current position for all rows
            df.at[idx, 'position'] = position
            df.at[idx, 'equity'] = equity

        self.logger.info("Backtest completed with single-trade handling.")
        return df


In [ ]:

class TestBacktest(unittest.TestCase):
    def setUp(self):
        """
        Set up mock data and a strategy instance for testing.
        """
        self.config = StrategyConfig()
        self.logger = MagicMock()
        self.strategy = Strategy(config=self.config, logger=self.logger)

        # Mock DataFrame with 60 rows
        self.df = pd.DataFrame({
            'close': [100.0] * 60,
            'high': [101.0] * 60,
            'low': [99.0] * 60,
            'volume': [1000] * 60,
        })
        self.df.loc[[14, 28, 42, 56], 'close'] = [100.0, 96.0, 100.0, 96.0]

    def test_backtest_strategy_multiple_trades(self):
        """
        Test backtest_strategy with multiple BUY and SELL signals.
        """
        df_with_indicators = self.strategy.calculate_indicators(self.df.copy())

        # Simulate multiple BUY and SELL signals
        signals = [TradeSignal.HOLD] * len(df_with_indicators)
        signals[14] = TradeSignal.BUY  # Enter LONG
        signals[28] = TradeSignal.SELL  # Exit LONG, then enter SHORT
        signals[42] = TradeSignal.BUY  # Exit SHORT, then enter LONG
        signals[56] = TradeSignal.SELL  # Exit LONG, then enter SHORT

        df_with_indicators['signal'] = signals
        result = self.strategy.backtest_strategy(df_with_indicators)

        # Validate positions and resets
        self.assertEqual(result.loc[14, 'position'], 1, "Should be LONG at index 14.")
        self.assertEqual(result.loc[28, 'position'], -1, "Should exit LONG and enter SHORT at index 28.")
        self.assertEqual(result.loc[42, 'position'], 1, "Should exit SHORT and enter LONG at index 42.")
        self.assertEqual(result.loc[56, 'position'], -1, "Should exit LONG and enter SHORT at index 56.")

        # Validate entry and exit prices
        self.assertEqual(result.loc[14, 'entry_price'], 100.0, "Entry price should be set at index 14.")
        self.assertEqual(result.loc[28, 'exit_price'], 96.0, "Exit price should match close price at index 28.")
        self.assertEqual(result.loc[42, 'entry_price'], 100.0, "Entry price should be set again at index 42.")
        self.assertEqual(result.loc[56, 'exit_price'], 96.0, "Exit price should match close price at index 56.")

        # Validate equity updates
        self.assertGreater(result.loc[28, 'equity'], 0, "Equity should update correctly after exit at index 28.")
        self.assertGreater(result.loc[56, 'equity'], 0, "Equity should update correctly after exit at index 56.")

        # Print debugging data
        print(result[['signal', 'position', 'entry_price', 'exit_price', 'strategy_returns', 'equity']].iloc[[14, 28, 42, 56]])

if __name__ == '__main__':
    unittest.main()


usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument 'c:\\Users\\USER\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3c0e47f59568c2e7c999a8caed1e9ea556d42ae0c.json'


SystemExit: 2

c:\Users\USER\anaconda3\envs\TradingRobotPlug\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:

suite = unittest.TestLoader().loadTestsFromTestCase(TestBacktest)
unittest.TextTestRunner().run(suite)
    

C:\Users\USER\AppData\Local\Temp\ipykernel_22420\1438446693.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[100.0, 96.0, 100.0, 96.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.df.loc[[14, 28, 42, 56], 'close'] = [100.0, 96.0, 100.0, 96.0]
INFO:TestBacktest:Starting backtest with single-trade handling.
DEBUG:TestBacktest:Entered LONG at index 14
DEBUG:TestBacktest:Exited LONG at index 28 with equity 99996.0
DEBUG:TestBacktest:Entered SHORT at index 28
DEBUG:TestBacktest:Exited SHORT at index 42 with equity 99992.0
DEBUG:TestBacktest:Entered LONG at index 42
DEBUG:TestBacktest:Exited LONG at index 56 with equity 99988.0
DEBUG:TestBacktest:Entered SHORT at index 56
INFO:TestBacktest:Backtest completed with single-trade handling.
F
FAIL: test_backtest_strategy_multiple_trades (__main__.TestBacktest.test_backtest_strategy_multiple_trades)
------------

   signal  position  entry_price  exit_price  strategy_returns    equity
14    BUY         1        100.0         0.0               0.0  100000.0
28   SELL        -1         96.0        96.0              -4.0   99996.0
42    BUY         1        100.0       100.0              -4.0   99992.0
56   SELL        -1         96.0        96.0              -4.0   99988.0


<unittest.runner.TextTestResult run=1 errors=0 failures=1>